# Lending Loop Web App
This is a prototype for a Lending Loop web app / dashboard. This notebook focuses on the logics of Lending Loop summary calculations.

# System Initializations

### Package Initializations

In [1]:
# DataFrames
import pandas as pd
import findspark
import pyspark
import pyspark.sql.functions as F
import pyspark.sql.types as T

# Numerical Packages
import numpy as np
import math

# Datetime
import datetime

# Misc Packages
import os

# Visualizations
import plotly
import plotly.offline as pyo
import plotly.graph_objs as go

### System Configuration


In [2]:
# Plotly offline
pyo.init_notebook_mode(connected=True)

# Pandas display
pd.set_option('display.max_colwidth', -1)

# PySpark Session Initialization
packages = 'org.mongodb.spark:mongo-spark-connector_2.11:2.2.0'
dedicated_memory = '1g'

os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages {} --driver-memory {} pyspark-shell' \
    .format(packages, dedicated_memory)

In [3]:
# Find SPARK_HOME
findspark.init()

# Create SparkSession
spark = (pyspark.sql.SparkSession
         .builder
         .appName('LendingLoop')
         .getOrCreate())

### Constant Definitions

In [4]:
LENDING_LOOP_FEE_RATE = 1.5

### Helper Function Definitions

In [5]:
def display_DF(sparkDF, n=5):
    '''
    Interactively displays the first n rows of a sparkDF as a pandas dataframe
    '''
    return (sparkDF
            .limit(n)
            .drop('_id')
            .toPandas())

def import_new_notes(DF):
    '''
    Search DF for new notes and import them into MongoDB.
    '''
    @F.udf(returnType=T.DoubleType())
    def calculate_principal(unitPay, interestRate, totalPayCycles):
        '''
        Calculate the principal invested based on the unitPay, interestRate and totalPayCycles.
        '''
        interestRate /= 100 * 12

        return unitPay / interestRate * (1 - 1. / (1 + interestRate)**totalPayCycles)
    
    @F.udf(returnType=T.FloatType())
    def bad_debt_funds(principal, grade, cyclesTotal):
        rate = interestRatesBroadcast.value[grade] / 1200
        return round(principal * (cyclesTotal * rate / (1 - (1 + rate)**(-cyclesTotal)) - 1), 2)
    
    # Obtain list of new notes from input DF
    newNotesDF = (DF
                  .orderBy('dueDate')
                  .groupBy('loanID', 'company', 'loanName', 'interestRate', 'grade')
                  .agg(F.count('loanID').alias('cyclesTotal'), 
                       F.sum('principalScheduled').alias('principal'),
                       F.round(F.sum('fees'), 2).alias('fees'),
                       F.round(F.sum('interestScheduled') - F.sum('fees'), 2).alias('profits'),
                       F.round(F.mean('totalScheduled'), 2).alias('unitPayment'), 
                       F.add_months(F.first('dueDate'), -1).alias('startDate'))
                  .withColumn('principal', 
                              F.round(calculate_principal('unitPayment', 'interestRate', 'cyclesTotal'), 0))
                  .withColumn('cyclesRemaining', F.udf(lambda x: x, T.LongType())('cyclesTotal'))
                  .withColumn('badDebtFunds', bad_debt_funds('principal', 'grade', 'cyclesTotal'))
                  .withColumn('profits', F.col('profits') - F.col('badDebtFunds'))
                  .withColumn('amountRepayed', F.lit(0.00))
                  .cache())
    
    return newNotesDF

# Test Area

### Import dataset

In [13]:
# Read raw CSV file
rawDF = (spark
         .read
         .format('com.databricks.spark.csv')
         .option('header', 'True')
         .option('inferschema', 'True')
         .load('all_payments.csv'))

# Camel case titles
camelCaseDict = {title: title[0].lower() + title.replace(' ', '')[1:] for title in rawDF.columns}

# Simplify certain column titles
camelCaseDict['Fees Paid to Loop'] = 'fees'
camelCaseDict['Risk Band'] = 'grade'
camelCaseDict['Loan Id'] = 'loanID'

# Camelcase column titles
rawDF = rawDF.select([F.col(title).alias(camelCaseDict[title]) for title in camelCaseDict.keys()])

In [31]:
testDF = pd.read_csv('https://www.dropbox.com/s/1zrh1636odu7757/all_payments.csv?dl=1')

In [10]:
schema = T.StructType([
    (T.StructField('Payment Type', T.StringType())),
    (T.StructField('Loan Id', T.IntegerType())),
    (T.StructField('Company', T.StringType())),
    (T.StructField('Loan Name', T.StringType())),
    (T.StructField('Interest Rate', T.FloatType())),
    (T.StructField('Risk Band', T.StringType())),
    (T.StructField('Interest Scheduled', T.FloatType())),
    (T.StructField('Principal Scheduled', T.FloatType())),
    (T.StructField('Total Scheduled', T.FloatType())),
    (T.StructField('Interest Owed', T.FloatType())),
    (T.StructField('Principal Owed', T.FloatType())),
    (T.StructField('Total Owed', T.FloatType())),
    (T.StructField('Interest Paid', T.FloatType())),
    (T.StructField('Principal Paid', T.FloatType())),
    (T.StructField('Total Paid', T.FloatType())),
    (T.StructField('Fees Paid to Loop', T.FloatType())),
    (T.StructField('Due Date', T.StringType())),
    (T.StructField('Date Paid', T.StringType())),
    (T.StructField('Status', T.StringType())),
])

rawDF = spark.createDataFrame(spark.sparkContext.textFile('https://www.dropbox.com/s/1zrh1636odu7757/all_payments.csv'), schema)
# Read raw CSV file
rawDF = (spark
         .read
         .format('csv')
         .option('header', 'True')
         .option('url', 'https://www.dropbox.com/s/1zrh1636odu7757/all_payments.csv?dl=1')
         .schema(schema)
         .load())

# Camel case titles
camelCaseDict = {title: title[0].lower() + title.replace(' ', '')[1:] for title in rawDF.columns}

# Simplify certain column titles
camelCaseDict['Fees Paid to Loop'] = 'fees'
camelCaseDict['Risk Band'] = 'grade'
camelCaseDict['Loan Id'] = 'loanID'

# Camelcase column titles
rawDF = rawDF.select([F.col(title).alias(camelCaseDict[title]) for title in camelCaseDict.keys()])

In [12]:
testDF = spark.createDataFrame(testDF, schema)
display_DF(testDF)

,Payment Type,Loan Id,Company,Loan Name,Interest Rate,Risk Band,Interest Scheduled,Principal Scheduled,Total Scheduled,Interest Owed,Principal Owed,Total Owed,Interest Paid,Principal Paid,Total Paid,Fees Paid to Loop,Due Date,Date Paid,Status
0,Scheduled Payment,59705,Nice and Smooth Ultramedia Inc.,Concert Producer & Promoter,20.67,D,0.86,2.39,3.25,0.86,2.39,3.25,0.86,2.39,3.25,0.06,2017-09-22,2017-09-23,Paid
1,Scheduled Payment,83910,Bronze Baxx Tanning Studio Inc.,Salon - Equipment Purchase (1 of 2),10.33,B+,0.64,2.83,3.47,0.64,2.83,3.47,0.64,2.83,3.47,0.09,2017-09-22,2017-09-22,Paid
2,Scheduled Payment,7802,Brightpath Capital Corporation,Mortgage Lender - Working Capital,10.33,B+,0.64,1.27,1.91,0.64,1.27,1.91,0.64,1.27,1.91,0.09,2017-09-25,2017-09-25,Paid
3,Scheduled Payment,96625,Rossco's Tree Service and contracting Ltd.,Arborist - Lease Buyouts (1 of 3),16.18,C,1.01,0.82,1.83,1.01,0.82,1.83,1.01,0.82,1.83,0.09,2017-09-30,2017-10-06,Paid
4,Scheduled Payment,73245,IT MULTISERVICES MG&CG INC.,E-commerce Company - Inventory Purchase,16.18,C,0.33,0.89,1.22,0.33,0.89,1.22,0.00,0.00,0.00,0.03,2017-10-07,NaN,Scheduled


In [14]:
display_DF(rawDF)

,status,interestPaid,paymentType,principalScheduled,totalOwed,loanID,grade,interestRate,company,principalPaid,dueDate,interestScheduled,totalScheduled,fees,loanName,principalOwed,totalPaid,interestOwed,datePaid
0,Paid,0.86,Scheduled Payment,2.39,3.25,59705,D,20.67,Nice and Smooth Ultramedia Inc.,2.39,2017-09-22,0.86,3.25,0.06,Concert Producer & Promoter,2.39,3.25,0.86,2017-09-23
1,Paid,0.64,Scheduled Payment,2.83,3.47,83910,B+,10.33,Bronze Baxx Tanning Studio Inc.,2.83,2017-09-22,0.64,3.47,0.09,Salon - Equipment Purchase (1 of 2),2.83,3.47,0.64,2017-09-22
2,Paid,0.64,Scheduled Payment,1.27,1.91,7802,B+,10.33,Brightpath Capital Corporation,1.27,2017-09-25,0.64,1.91,0.09,Mortgage Lender - Working Capital,1.27,1.91,0.64,2017-09-25
3,Paid,1.01,Scheduled Payment,0.82,1.83,96625,C,16.18,Rossco's Tree Service and contracting Ltd.,0.82,2017-09-30,1.01,1.83,0.09,Arborist - Lease Buyouts (1 of 3),0.82,1.83,1.01,2017-10-06
4,Paid,0.33,Scheduled Payment,0.89,1.22,73245,C,16.18,IT MULTISERVICES MG&CG INC.,0.89,2017-10-07,0.33,1.22,0.03,E-commerce Company - Inventory Purchase,0.89,1.22,0.33,2017-10-07


### Initialise Base `Summary` State

In [15]:
# # TODO Update to realistic values
# baseState = {
#     'lifeTimeEarnings': 0.,
#     'netROI': 0.,
#     'availableFunds': 1500,
#     'fundsInvested': 0.,
#     'fundCommited': 0.,
#     'lastUpdated': datetime.datetime(2017,9,1),
# }

### Add new notes to collection

In [16]:
# import_new_notes(rawDF)

### Define Grades interest Rate

In [17]:
interestRates = {
    'A+': 0.56,
    'A': 1.83,
    'B+': 3.05,
    'B': 4.32,
    'C+': 5.46,
    'C': 6.25,
    'D+': 7.11,
    'D': 8.00,
    'E+': 8.84,
    'E': 9.64,
}

# Broadcast interest rates
interestRatesBroadcast = spark.sparkContext.broadcast(interestRates)

### Pull new collection

In [18]:
notesDF = import_new_notes(rawDF)
display_DF(notesDF)

,loanID,company,loanName,interestRate,grade,cyclesTotal,principal,fees,profits,unitPayment,startDate,cyclesRemaining,badDebtFunds,amountRepayed
0,7802,Brightpath Capital Corporation,Mortgage Lender - Working Capital,10.33,B+,48,75.0,2.45,9.44,1.91,2017-08-25,48,4.76,0.0
1,83910,Bronze Baxx Tanning Studio Inc.,Salon - Equipment Purchase (1 of 2),10.33,B+,24,75.0,1.21,4.59,3.47,2017-08-22,24,2.41,0.0
2,90873,Nitin Chauhan Medicine Professional Corporation,"Head, Neck & Facial Plastic Surgeon - Working Capital",8.52,A,24,50.0,0.81,2.66,2.27,2017-09-07,24,0.96,0.0
3,75251,Pops Pub Ltd.,Local Pub - Refinancing and Working Capital,10.33,B+,24,50.0,0.81,3.02,2.31,2017-10-06,24,1.60,0.0
4,67770,Jenco Canada Inc.,LED Lighting Distributor and Franchisor,8.52,A,12,50.0,0.42,1.38,4.36,2017-09-07,12,0.50,0.0


### Import New Transactions

In [19]:
def update_state(rawDF, notesDF):
    '''
    Update state with new transactions from DF.
    '''
    # Obtain transactions that have occured
    netTransactionsDF = (rawDF
                         .filter(F.isnull('datePaid') == 'False')
                         .groupBy('loanID')
                         .agg(F.sum('totalPaid').alias('totalPaid')))
    
    # Update noteStates
    updatedNotesStateDF = (notesDF
                           .join(netTransactionsDF, 'loanID', 'left_outer')
                           .withColumn('amountRepayed', F.col('amountRepayed') + F.col('totalPaid'))
                           .withColumn('cyclesRemaining', F.col('cyclesRemaining') - 1)
                           .drop('totalPaid')
                           .fillna(0.0)
                           .cache())
    
    return updatedNotesStateDF

In [20]:
notesDF = update_state(rawDF, notesDF)
display_DF(notesDF, 10)

,loanID,company,loanName,interestRate,grade,cyclesTotal,principal,fees,profits,unitPayment,startDate,cyclesRemaining,badDebtFunds,amountRepayed
0,7802,Brightpath Capital Corporation,Mortgage Lender - Working Capital,10.33,B+,48,75.0,2.45,9.44,1.91,2017-08-25,47,4.76,1.91
1,83910,Bronze Baxx Tanning Studio Inc.,Salon - Equipment Purchase (1 of 2),10.33,B+,24,75.0,1.21,4.59,3.47,2017-08-22,23,2.41,3.47
2,90873,Nitin Chauhan Medicine Professional Corporation,"Head, Neck & Facial Plastic Surgeon - Working Capital",8.52,A,24,50.0,0.81,2.66,2.27,2017-09-07,23,0.96,2.27
3,75251,Pops Pub Ltd.,Local Pub - Refinancing and Working Capital,10.33,B+,24,50.0,0.81,3.02,2.31,2017-10-06,23,1.60,0.00
4,67770,Jenco Canada Inc.,LED Lighting Distributor and Franchisor,8.52,A,12,50.0,0.42,1.38,4.36,2017-09-07,11,0.50,4.36
5,96625,Rossco's Tree Service and contracting Ltd.,Arborist - Lease Buyouts (1 of 3),16.18,C,60,75.0,3.23,18.83,1.83,2017-08-31,59,12.52,1.83
6,83061,J.K. ENGINEERING LTD,Infrastructure Engineering and Construction Firm - Second Loan,8.52,A,36,100.0,2.42,8.21,3.15,2017-09-15,35,2.85,0.00
7,73245,IT MULTISERVICES MG&CG INC.,E-commerce Company - Inventory Purchase,16.18,C,24,25.0,0.41,2.22,1.22,2017-09-07,23,1.66,1.22
8,36403,Social Theorem Consulting Inc,Digital Media Agency - Refinancing And Continued Expansion,18.40,D+,36,25.0,0.63,4.06,0.90,2017-09-07,35,2.83,0.91
9,45851,The Extensionist Consultancy Inc,Salon Chain - Inventory Purchase,12.20,B,36,75.0,1.84,7.80,2.49,2017-09-14,35,5.10,0.00


### Create custom progress bar

In [38]:
def analyze_notes(DF):
    '''
    Analyze notes by calculating payment breakdowns based on pre-specified categories.
    '''
    # Define helper function
    def _min_max(x, minVal, maxVal):
        assert minVal <= maxVal
        return max(minVal, min(x, maxVal))
    
    def _capitalise_first_letter(string):
        return ''.join([char.upper() if i == 0 else char for i, char in enumerate(string)])
    
    # Define udfs
    @F.udf(returnType=T.FloatType())
    def calc_received(val, maxVal, *args):
        return _min_max(val - sum(args), 0., maxVal)
    
    @F.udf(returnType=T.FloatType())
    def calc_next_payment(val, maxVal, unitPayment, *args):
        # Handle case when nextPMT spills over from previous category
        if (val - sum(args) < 0):
            return _min_max(unitPayment - (sum(args) - val), 0., unitPayment)
        
        remainder = _min_max(maxVal - (val - sum(args)), 0., maxVal)
        return min(remainder, unitPayment)
    
    @F.udf(returnType=T.FloatType())
    def calc_outstanding(val, maxVal, unitPayment, *args):
        return _min_max(maxVal - (val + unitPayment - sum(args)), 0., maxVal)
    
    @F.udf(returnType=T.IntegerType())
    def months_to_breakeven(nextPrincipalPayment, principalOutstanding, unitPayment):
        return int(math.ceil((nextPrincipalPayment + principalOutstanding) / unitPayment))
    
    @F.udf(returnType=T.BooleanType())
    def completed(cyclesRemaining):
        return not cyclesRemaining
        
    
    # Add relevant columns to notesDF
    categories = ['fees', 'principal', 'badDebtFunds', 'profits']
    
    # Add `received`, `next_payment` and `outstanding` columns programmatically
    for i, category in enumerate(categories):
        DF = (DF
              .withColumn('{}Received'.format(category) if category != 'fees' else 'feesPaid', 
                          calc_received('amountRepayed', category, *categories[:i]))
              .withColumn('next{}Payment'.format(_capitalise_first_letter(category)), 
                          calc_next_payment('amountRepayed', category, 'unitPayment', *categories[:i]))
              .withColumn('{}Outstanding'.format(category), 
                          calc_outstanding('amountRepayed', category, 'unitPayment', *categories[:i])))
        
    # Add `monthsToBreakeven` and `completed` column
    DF = (DF
          .withColumn('monthsToBreakeven', 
                      months_to_breakeven('nextPrincipalPayment',
                                          'principalOutstanding',
                                          'unitPayment'))
          .withColumn('completed', completed('cyclesRemaining'))
          .orderBy(['feesPaid', 'principal', 'principalOutstanding', 'interestRate', 'cyclesTotal'], 
                   ascending=[1, 1, 0, 1, 1])
          .cache())
    
    return DF

In [39]:
analyzedNotesDF = analyze_notes(notesDF)
display_DF(analyzedNotesDF)

,loanID,company,loanName,interestRate,grade,cyclesTotal,principal,fees,profits,unitPayment,...,nextPrincipalPayment,principalOutstanding,badDebtFundsReceived,nextBadDebtFundsPayment,badDebtFundsOutstanding,profitsReceived,nextProfitsPayment,profitsOutstanding,monthsToBreakeven,completed
0,87799,1005351 BC LTD. O/A Liberty Tax Service,Liberty Tax - Territory Expansion,10.33,B+,48,25.0,0.82,2.99,0.63,...,0.00,25.000000,0.0,0.0,1.59,0.0,0.0,2.99,40,False
1,4370,631343 alberta ltd,Denture Clinic - Leasehold Improvements and Marketing,14.11,C+,36,25.0,0.62,2.83,0.85,...,0.23,24.770000,0.0,0.0,2.16,0.0,0.0,2.83,30,False
2,75251,Pops Pub Ltd.,Local Pub - Refinancing and Working Capital,10.33,B+,24,50.0,0.81,3.02,2.31,...,1.50,48.500000,0.0,0.0,1.60,0.0,0.0,3.02,22,False
3,29560,NuEnergy Systems Inc.,Integrated Solar Energy Services Company,6.80,A+,36,75.0,1.78,5.51,2.30,...,0.52,74.480003,0.0,0.0,0.65,0.0,0.0,5.51,33,False
4,74358,SCG Sign City Group Inc,Exterior Construction Company,10.33,B+,36,75.0,1.82,6.98,2.43,...,0.61,74.389999,0.0,0.0,3.58,0.0,0.0,6.98,31,False


In [90]:
def progress_bar(DF):
    '''
    Creates a custom progress bar based on analyzedNotesDF.
    '''
    def _convert_to_list(colName):
        '''
        Extract the colName of notesDF as a list, with each entry rounded to 2 decimal places.
        '''
        return (DF
                .select(colName)
                .rdd
                .map(lambda x: x[0] if colName == 'company' or colName == 'monthsToBreakeven' else round(x[0], 2))
                .collect())
    
    def _create_trace(colName):
        '''
        Create a trace based on colName.
        '''
        colourDict = {
            'feesPaid': '#585858', 
            'nextFeesPayment': '#848484', 
            'feesOutstanding': '#BDBDBD', 
            'principalReceived': '#512361',
            'nextPrincipalPayment': '#9165AE',
            'principalOutstanding': '#C2B2C8',
            'badDebtFundsReceived': '#FF8000',
            'nextBadDebtFundsPayment': '#FAAC58',
            'badDebtFundsOutstanding': '#F5D0A9',
            'profitsReceived': '#72C02C', 
            'nextProfitsPayment': '#a2de6e',
            'profitsOutstanding': '#d7f1c1'
        }
        
        names = {
            'feesPaid': 'Fees Paid', 
            'nextFeesPayment': 'Next Fee Payment', 
            'feesOutstanding': 'Fees Outstanding', 
            'principalReceived': 'Principal Received',
            'nextPrincipalPayment': 'Next Principal Payment',
            'principalOutstanding': 'Principal Outstanding', 
            'badDebtFundsReceived': 'Bad Debt Funds Received',
            'nextBadDebtFundsPayment': 'Next Bad Debt Funds Payment',
            'badDebtFundsOutstanding': 'Bad Debt Funds Outstanding',
            'profitsReceived': 'Profits Received', 
            'nextProfitsPayment': 'Next Profit Payment',
            'profitsOutstanding': 'Profits Outstanding'
        }
        
        groups = {
            'feesPaid': 'Fees', 
            'nextFeesPayment': 'Fees', 
            'feesOutstanding': 'Fees',  
            'principalReceived': 'Principal',
            'nextPrincipalPayment': 'Principal',
            'principalOutstanding': 'Principal', 
            'badDebtFundsReceived': 'Bad Debt Funds',
            'nextBadDebtFundsPayment': 'Bad Debt Funds',
            'badDebtFundsOutstanding': 'Bad Debt Funds',
            'profitsReceived': 'Profits', 
            'nextProfitsPayment': 'Profits',
            'profitsOutstanding': 'Profits'
        }
        
        return go.Bar(
            x = _convert_to_list(colName),
            y = companiesList,
            name = names[colName],
            orientation = 'h',
            width = 0.75,
            hoverinfo = 'text',
            text = ['' if val == 0 else '${}'.format(val) for val in _convert_to_list(colName)],
            legendgroup = groups[colName],
            marker = {
                'color': colourDict[colName],
                'line': {'width': 0.}
            }
        )
    
    def _capitalise_first_letter(string):
        return ''.join([char.upper() if i == 0 else char for i, char in enumerate(string)])
    
    # Obtain list of companies
    companiesList = _convert_to_list('company')
    
    # Add relevant columns to notesDF
    categories = ['fees', 'principal', 'badDebtFunds', 'profits']
    
    # Create keys for traces
    _megaList = [['{}Received'.format(category) if category != 'fees' else 'feesPaid', 
                 'next{}Payment'.format((_capitalise_first_letter(category))), 
                 '{}Outstanding'.format(category)] for category in categories]
    
    keys = [item for sublist in _megaList for item in sublist]
    1
    # Create traces
    traces = []
    for key in keys:
        traces.append(_create_trace(key))
    data = go.Data(traces)
    
    # Create annotations
    annotationData = (DF
                      .select('company', 'principal', 'monthsToBreakeven')
                      .collect())
    annotations = [{
            'x': row['principal'] - 2,
            'y': row['company'],
            'text': row['monthsToBreakeven'] if row['monthsToBreakeven'] > 0 else '',
            'font': {
                'family': 'Arial',
                'size': 14,
                'color': 'rgb(255, 255, 255)'},
            'showarrow': False
        } for row in annotationData]
    
    # Create layout
    layout = go.Layout(
        height = 30 * DF.count() + 280,
        xaxis = {
            'domain': [0.2, 1],
            'tickprefix': '$'
        },
        barmode = 'stack',
        legend = {
            'orientation': 'h',
            'traceorder': 'grouped'
        },
        annotations = annotations,
        margin = {'t': 0}
    )
    
    figure = go.Figure(data=data, layout=layout)
    
    pyo.iplot(figure)

In [91]:
progress_bar(analyzedNotesDF)

In [25]:
def wind_rose(DF):
    '''
    Visualise principal invested in notes by grades.
    '''
    
    grades = ['A+', 'A', 'B+', 'B', 'C+', 'C', 'D+', 'D', 'E+', 'E']
    fundsDict = (DF
                 .groupBy('grade')
                 .agg(F.sum('principal'))
                 .rdd
                 .collectAsMap())
    
    trace = go.Area(
        r = [fundsDict[grade] if grade in fundsDict.keys() else 0 for grade in grades],
        t = grades,
        marker = {'color': '#9165AE'},
    )
    
    data = go.Data([trace])
    
    layout = go.Layout(
        orientation = -90,
        angularaxis = {
            'showticklabels': False,
            'showline': False,
        },
        radialaxis = {
            'orientation': -90,
            'visible': False,
            'showline': False
        },
        showlegend = False,
    )
    
    pyo.iplot(go.Figure(data=data, layout=layout))
    
wind_rose(notesDF)

In [85]:
def notes_distribution_bar_chart(DF):
    '''
    Plot a bar chart showing the fund distribution in each note grade.
    '''
    grades = ['A+', 'A', 'B+', 'B', 'C+', 'C', 'D+', 'D', 'E+', 'E']
    fundsDict = (DF
                 .groupBy('grade')
                 .agg(F.sum('principal'))
                 .rdd
                 .collectAsMap())
    
    trace = go.Bar(
        x = grades,
        y = [fundsDict[grade] if grade in fundsDict.keys() else 0 for grade in grades],
        marker = {'color': '#9165AE'},
    )
    
    data = go.Data([trace])
    
    layout = go.Layout(
        yaxis = {'tickprefix': '$'},
        margin = {
            't': 0,
        }
    )
    
    pyo.iplot(go.Figure(data=data, layout=layout))

notes_distribution_bar_chart(notesDF)

In [37]:
display_DF(analyzedNotesDF)

,loanID,company,loanName,interestRate,grade,cyclesTotal,principal,fees,profits,unitPayment,...,principalReceived,nextPrincipalPayment,principalOutstanding,badDebtFundsReceived,nextBadDebtFundsPayment,badDebtFundsOutstanding,profitsReceived,nextProfitsPayment,profitsOutstanding,monthsToBreakeven
0,87799,1005351 BC LTD. O/A Liberty Tax Service,Liberty Tax - Territory Expansion,10.33,B+,48,25.0,0.82,2.99,0.63,...,0.0,0.00,25.000000,0.0,0.0,1.59,0.0,0.0,2.99,40
1,4370,631343 alberta ltd,Denture Clinic - Leasehold Improvements and Marketing,14.11,C+,36,25.0,0.62,2.83,0.85,...,0.0,0.23,24.770000,0.0,0.0,2.16,0.0,0.0,2.83,30
2,75251,Pops Pub Ltd.,Local Pub - Refinancing and Working Capital,10.33,B+,24,50.0,0.81,3.02,2.31,...,0.0,1.50,48.500000,0.0,0.0,1.60,0.0,0.0,3.02,22
3,29560,NuEnergy Systems Inc.,Integrated Solar Energy Services Company,6.80,A+,36,75.0,1.78,5.51,2.30,...,0.0,0.52,74.480003,0.0,0.0,0.65,0.0,0.0,5.51,33
4,74358,SCG Sign City Group Inc,Exterior Construction Company,10.33,B+,36,75.0,1.82,6.98,2.43,...,0.0,0.61,74.389999,0.0,0.0,3.58,0.0,0.0,6.98,31


In [48]:
(analyzedNotesDF
 .filter(F.col('completed') == False)
 .groupBy()
 .agg(F.round(F.sum('principalOutstanding'), 2).alias('principalOutstanding'), F.sum('badDebtFundsReceived').alias('badDebtFunds')).first())

Row(principalOutstanding=1184.46, badDebtFunds=0.0)

In [80]:
def funds_bar_chart(DF, fundsInvested):
    '''
    Create a horizontal bar chart based on funds invested and analyzedDF.
    '''
    # Compute categories
    principalOutStanding, badDebtFunds = (DF.filter(F.col('completed') == False)
                                          .groupBy()
                                          .agg(F.round(F.sum('principalOutstanding'), 2).alias('principalOutstanding'), 
                                               F.sum('badDebtFundsReceived').alias('badDebtFunds'))
                                          .first())
    fundsAvailable = fundsInvested - principalOutStanding - badDebtFunds
    
    # Create dictionary
    funds = {
        'Principal Outstanding': (principalOutStanding, '#9165AE'),
        'Bad Debt Fund': (badDebtFunds, '#FF8000'),
        'Funds Available': (fundsAvailable, '#00BBFF')
    }
    
    # Create traces
    traces = []
    
    for fundName in ['Funds Available', 'Bad Debt Fund', 'Principal Outstanding']:
        fund, colour = funds[fundName]
        
        traces.append(go.Bar(
            x = [fund],
            marker = {'color': colour},
            hoverinfo = 'text',
            text = ['' if fund == 0 else '${:.2f}<br>{}'.format(fund, fundName)]
        ))
        
    data = go.Data(traces)
    
    layout = go.Layout(
        height = 50,
        xaxis = {
            'showline': False,
            'showgrid': False,
            'showticklabels': False,
            'zeroline': False
        },
        yaxis = {
            'showline': False,
            'showgrid': False,
            'showticklabels': False,
        },
        barmode = 'stack',
        showlegend = False,
        margin = {
            't': 0,
            'b': 0
        }
    )
    
    figure = go.Figure(data=data, layout=layout)
    
    pyo.iplot(figure)


funds_bar_chart(analyzedNotesDF, 1500)

In [27]:
def calc_total_earnings(DF):
    '''
    Calculate total earnings based on analyzedNotesDF since a certain date. 
    '''
    # TODO: Make result dependent on dates
    # if since != None:
        # assert isinstance(since, datetime.datetime)
        # DF = DF.filter()
    return (DF
            .select((F.col('badDebtFundsReceived') + F.col('profitsReceived')).alias('earnings'))
            .groupBy()
            .agg(F.sum('earnings')).first()[0])

calc_total_earnings(analyzedNotesDF)

0.0

In [28]:
def calc_ROI(DF):
    result = (DF
            .select((F.col('badDebtFundsReceived') + F.col('profitsReceived')).alias('earnings'),
                    'principal')
            .groupBy()
            .agg(F.sum('earnings').alias('earnings'), F.sum('principal').alias('principal')).first())
    
    return result['earnings'] / result['principal']

calc_ROI(analyzedNotesDF)

0.0

In [29]:
def calc_diversification(DF):
    '''
    Diversification is calculated as per Lending Loop's dashboard: maximum investment divided by total investment.
    '''
    result = (DF
              .groupBy()
              .agg(F.sum('principal').alias('totalPrincipal'), 
                   F.max('principal').alias('maxPrincipal'))
              .first())
    return round(result['maxPrincipal'] / result['totalPrincipal'] * 100, 1)

calc_diversification(notesDF)

8.2

In [30]:
def calc_funds_remaining(DF, totalFunds):
    return totalFunds - DF.groupBy().agg(F.sum('principal')).first()[0]

calc_funds_remaining(notesDF, 1500)

275.0

### TODO
- Design dashboard
    - Vertical bar chart showing distribution of funds
    - Change between different time periods
    - Wind rose showing distributino of funds for different grades
- Predictor bar chart showing expected returns if invest in new funds

In [ ]:
testDF = spark.createDataFrame([(6.0, 'My Test Company', 3.0, 16.18, 75.0, 10., 36, 18.14, 5.),
                                (20.0, 'My Test Company1', 3.0, 16.18, 75.0, 10., 36, 18.14, 5.),
                                (70.0, 'My Test Company2', 3.0, 16.18, 75.0, 10., 36, 18.14, 5.),
                                (80.0, 'My Test Company3', 3.0, 16.18, 75.0, 10., 36, 18.14, 5.),
                                (115.0, 'My Test Company4', 3.0, 16.18, 75.0, 10., 36, 18.14, 5.)],
                               ['amountRepayed', 'company', 'feesAccrued', 'interestRate', 
                                'principal', 'unitPayment', 'cyclesTotal', 'profits', 'badDebtFunds'])

display_DF(testDF)